In [1]:
import networkx as nx
from itertools import combinations
import os
import re
import regex
import json
import lucene
from lupyne import engine
import json
import unicodedata
import os
import time

In [2]:
lucene.initVM(vmargs=['-Djava.awt.headless=true'])

In [3]:
indexer = engine.Indexer('lucene_index')

In [4]:
def load_graph():
    try:
        return nx.read_gml('graph_index.gml')
    except FileNotFoundError or nx.NetworkXError:
        loaded_graph = False
        return nx.Graph()

In [5]:
G = load_graph()

In [6]:
def get_connections_if_any(query):
    result_query = query
    any_connections = re.search('connections:.*;', query)
    connections_entities_to_find = []
    if any_connections:
        result_query.replace(str(any_connections.group(
            0)), '')
        any_connections = any_connections.group(
            0).replace('connections:', '')
        entities = regex.findall('([a-zA-Z \.]*)', any_connections)
        connections_entities_to_find = [
            entity.strip() for entity in entities if entity != '']
    return connections_entities_to_find, result_query

In [7]:
def terminal_show_commands():
    print('If you want to exit program, them enter: exit')
    print("If you want find connections between entities write them in this format: connections: <Robert Fico>; <Peter Pellegrini>; ...")
    print('Enter query: ')

In [8]:
def run():
    Q = engine.Query
    while True:
        terminal_show_commands()
        command = input()
        if command == "exit":
            break
        else:
            start = time.time()
            connections_entities, query = get_connections_if_any(command)
            # q = Q.phrase('text', 'slovensko')
            hits = indexer.search(query, field='content', count=10, mincount=25000, timeout=30)
            if len(hits) == 0:
                print('We did not found anything relevant')
            else:
                print('\nFound results: \n')
                for i, hit in enumerate(hits):
                    with open('/usr/src//HTML/' + hit['file_path'], 'r', encoding='utf-8') as json_file:
                        doc = json.load(json_file)
                        print(doc['url'])
                        print(doc['title'])
                        print()
            neighbors = set()
            for entity in connections_entities:
                try:
                    neighbors.update(x[0] for x in sorted(
                        G[entity].items(), key=lambda edge: edge[1]['weight']))
                except KeyError:
                    pass
            if neighbors:
                print('\n', connections_entities, ': ', ', '.join(
                    list(neighbors - set(connections_entities))[:10]))
            end = time.time()
            print("Running time: ", str(end - start))

In [ ]:
run()

If you want to exit program, them enter: exit
If you want find connections between entities write them in this format: connections: <Robert Fico>; <Peter Pellegrini>; ...
Enter query: 


 threema



Found results: 

https://www.aktuality.sk/clanok/731921/rutime-sa-do-ery-emocionality-varuje-psychiater-podcast/
Rútime sa do éry emocionality, varuje psychiater (Podcast)

https://www.aktuality.sk/clanok/716088/ako-kocner-zachranoval-koaliciu-na-maldivach-s-bugarom-mimoriadny-podcast/
Ako Kočner zachraňoval koalíciu. Na Maldivách s Bugárom (mimoriadny podcast)

https://www.aktuality.sk/clanok/717775/merkelovu-zaujimal-kuciak-a-protesty-ako-kocner-a-zsuzsova-chceli-vyuzit-chudobnych-podcast/
Merkelovú zaujímal Kuciak a protesty. Ako Kočner a Zsuzsová chceli využiť chudobných (podcast)

https://www.aktuality.sk/clanok/744253/sudcovia-potvrdili-threemu-jankovska-a-kocner-sa-aj-stretli-sef-tiposu-v-putach-podcast/
Sudcovia potvrdili Threemu, Jankovská a Kočner sa aj stretli. Šéf Tiposu v putách (podcast)

https://www.aktuality.sk/clanok/786465/kocnerova-threema-ide-pred-sud-podcast/
Kočnerova Threema ide pred súd (podcast)

https://www.aktuality.sk/clanok/854940/dobehnu-televizne-zmenky-

 connections: Jan Kuciak;



Found results: 

https://www.aktuality.sk/clanok/714624/komentar-petra-bardyho-kde-vsade-sa-v-policii-este-vynori-kocner-a-boedoer/
Komentár Petra Bárdyho: Kde všade sa v polícii ešte vynorí Kočner a Bödör?

https://www.aktuality.sk/clanok/662966/komentar-daga-danisa-vrazda-lipsica-mala-stat-200-tisic-eur/
Komentár Daga Daniša: Vražda Lipšica mala stáť 200-tisíc eur

https://www.aktuality.sk/clanok/726475/komentar-daga-danisa-ako-sme-naleteli-kocnerovi/
Komentár Daga Daniša: Ako sme naleteli Kočnerovi

https://www.aktuality.sk/clanok/754753/komentar-petra-bardyho-priznal-sa-k-vrazde-ale-nie-je-v-tom-sam/
Komentár Petra Bárdyho: Priznal sa k vražde, ale nie je v tom sám

https://www.aktuality.sk/clanok/568612/komentar-daga-danisa-kalinaka-a-gaspara-treba-prezliect/
Komentár Daga Daniša: Kaliňáka a Gašpara treba prezliecť

https://www.aktuality.sk/clanok/899409/sud-mal-v-pripade-kuciak-urobit-chyby-nechape-ich-kuciak-starsi-ani-pravnik-kvasnica-podcast/
Súd mal v prípade Kuciak urobiť c

...


Found results: 

https://www.aktuality.sk/clanok/117806/bezdomovkyna-vypovedala-k-pokusu-o-dvojnasobnu-vrazdu/
Bezdomovkyňa vypovedala k pokusu o dvojnásobnú vraždu

https://www.aktuality.sk/clanok/73lqyfv/zlodejom-sa-v-rekreacnych-oblastiach-dari/
Zlodejom sa v rekreačných oblastiach darí

https://www.aktuality.sk/clanok/pgxn3vx/spravca-pritvrdil-basternakovci-mozu-prist-aj-o-luxusnu-vilu-pri-slavine/
Správca pritvrdil, Bašternákovci môžu prísť aj o luxusnú vilu pri Slavíne

https://www.aktuality.sk/clanok/593012/komentar-ronalda-izipa-velke-peniaze-stastie-neprinesu/
Komentár Ronalda Ižipa: Veľké peniaze šťastie neprinesú

https://www.aktuality.sk/clanok/518241/hlucny-koncert-na-teryho-chate-kritizuju-ho-iba-ti-ktori-tam-neboli/
Hlučný koncert na Téryho chate? Kritizujú ho iba tí, ktorí tam neboli

https://www.aktuality.sk/clanok/906539/remisova-chce-cez-vikend-v-divocine-kriesit-za-ludi-chata-to-nevyriesi-odkazuje-kolikova/
Remišová chce cez víkend v divočine kriesiť Za ľudí. Chata

 2021



Found results: 

https://www.aktuality.sk/clanok/e4455sy/statisticky-urad-ceny-95-oktanoveho-benzinu-lpg-a-nafty-vzrastli/
Štatistický úrad: Ceny 95-oktánového benzínu, LPG a nafty vzrástli

https://www.aktuality.sk/clanok/8y3egxs/prokurator-zrusil-obvinenie-novinarov-moniky-todovej-a-konstantina-cikovskeho/
Prokurátor zrušil obvinenie novinárov Moniky Tódovej a Konštantína Čikovského

https://www.aktuality.sk/clanok/6m5ksh6/eurostat-v-druhom-stvrtroku-2021-stupol-pocet-prvoziadatelov-o-azyl-v-eu/
Eurostat: V druhom štvrťroku 2021 stúpol počet prvožiadateľov o azyl v EÚ

https://www.aktuality.sk/clanok/6t70nhk/inspekcia-obvinila-dvoch-vysetrovatelov-z-timu-ocistec/
Inšpekcia obvinila dvoch vyšetrovateľov z tímu Očistec

https://www.aktuality.sk/clanok/898291/lahky-podcast-o-tazkej-korupcii-ako-je-to-v-skutocnosti-s-manipulaciami-vo-vysetrovani/


https://www.aktuality.sk/clanok/900826/koho-by-malo-najviac-zasiahnut-milionove-odskodne-pre-kuciakovcov-a-kusnirovcov/
Koho by malo najviac

 2030



Found results: 

https://www.aktuality.sk/clanok/548722/komentar-juraja-mesika-vcasne-varovanie-pre-novu-radu-vlady/
Komentár Juraja Mesíka: Včasné varovanie pre novú Radu vlády

https://www.aktuality.sk/clanok/537249/eu-na-ceste-k-cistej-mobilite/
EÚ na ceste k čistej mobilite

https://www.aktuality.sk/clanok/7jmyg4x/eurokomisia-predlozila-konkretny-plan-digitalnej-transformacie-eu-do-roku-2030/
Eurokomisia predložila konkrétny plán digitálnej transformácie EÚ do roku 2030

https://www.aktuality.sk/clanok/p3q8jyy/cina-rozsiruje-svoj-jadrovy-arzenal-rychlejsie-nez-sa-ocakavalo/
Čína rozširuje svoj jadrový arzenál rýchlejšie, než sa očakávalo

https://www.aktuality.sk/clanok/726806/caputova-v-new-yorku-chvalila-slovensko-naozaj-ma-za-co-podcast/
Čaputová v New Yorku chválila Slovensko – naozaj má za čo? (podcast)

https://www.aktuality.sk/clanok/687199/organizatorka-strajku-studentov/
Organizátorka štrajku študentov: Chceme zavrieť bane, zastaviť emisie a výrub lesa

https://www.aktual